# Import Statements

In [ ]:
import gurobipy as gp
import networkx as nx
from networkx.algorithms.approximation.steinertree import steiner_tree as steiner_tree_2
import random
from math import log
import scipy
from scipy.optimize import newton
from scipy.optimize import root_scalar
import sympy
from abc import ABC, abstractmethod

# Constants

## Debug Level Flags

In [ ]:
DEBUG_LEVEL_NONE = 0
DEBUG_LEVEL_THEORY_1 = 1.1
DEBUG_LEVEL_THEORY_2 = 1.2
DEBUG_LEVEL_FULL = 2
DEBUG_LEVEL_EXTREME = 3

## General Constants

In [ ]:
DEBUG_LEVEL = DEBUG_LEVEL_THEORY_1
NUM_NODES = 100
NUM_EDGES = 150
DEGREE = 3
NUM_MULTICAST_REQUESTS = 50
MAX_MULTICAST_SIZE = NUM_NODES//5
TOLERANCE = 0.1

NATURAL = 0
UNIFORM = 1
PERTURB = 2

# Helper Functions

In [ ]:
def dict_innerProd(x, y):
    assert x.keys() == y.keys()
    return sum(x[key]*y[key] for key in x.keys())

# Generating Problem Instances

In [ ]:
def is_connected(G):
    return nx.is_k_edge_connected(G, 1)

def get_random_connected_graph():
    G = nx.empty_graph(NUM_NODES)
    while not is_connected(G):
        G = nx.random_regular_graph(DEGREE, NUM_NODES)
    return G

class MulticastRequest:
    def __init__(self, size, graph, source=None, recipients=None):
        if source is None and recipients is None:
            multicast_group = random.sample(list(graph), size)
            source = multicast_group[0]
            recipients = set(multicast_group[1:])
        self.source = source
        self.recipients = recipients
        self.size = len(recipients) + 1
    
    def multicast_group(self):
        return self.recipients.union([self.source])

class MulticastPackingInstance:
    def __init__(self, num_requests, max_request_size, graph=None, requests=None):
        if graph is None:
            graph = get_random_connected_graph()
        if requests is None:
            requests = list()
            for i in range(num_requests):
                k = random.randint(2, max_request_size)
                requests.append(MulticastRequest(k, graph))
        self.graph = graph
        self.num_edges = len(self.graph.edges())
        self.requests = requests
        self.num_requests = len(requests)

# Jansen-Zhang General Convex min-max

In [ ]:
def theta_eq(theta, x, t, M, f):
    retval = 1
    for e in self.instance.graph.edges():
        retval = retval - t*theta/(M*(theta - f(x,e)))
    print("theta = {}".format(theta))
    print("theta_eq = {}".format(retval))
    return retval

def derivative_theta_eq(theta, x, t, M, f):
    retval = 0
    for e in self.instance.graph.edges():
        retval = retval + t*f(x,e)/(M*(theta - f(x,e))*(theta - f(x,e)))
    print("d_theta_eq = {}".format(retval))
    return retval

class JansenZhangMinMaxer:
    sigma0 = 1
    
    def __init__(self, constraint_indices, f, approx_block_solver):
        self.constraint_indices = constraint_indices
        self.M = len(constraint_indices)
        self.f = f
        self.abs = approx_block_solver
        self.current_solution = None
        self.lamb = dict()
        self.t = self.sigma0/6
        self.prices = dict()
        self.theta = dict()
    
    # Define a function that computes theta_t(x)
    def theta_via_sympy(self, x, t):
        M = self.M
        f = self.f
        
        theta = sympy.Symbol('theta')
        expression = 1
        for m in constraint_indices:
            expression -= (t/M) * theta/(theta - f(x,m))
        soln_set = sympy.solvers.solveset(expression, theta, domain=sympy.Interval(self.lamb[x], sympy.S.Infinity, True, True))
        #assert len(soln_set == 1)
        if DEBUG_LEVEL >= DEBUG_LEVEL_FULL:
            print(soln_set)
        for soln in soln_set:
            retval = soln
        return retval
        
    def calculate_theta(x, t):
        return scipy.optimize.brentq(theta_eq, self.lamb[x]/(1-t/self.M), self.lamb[x]/(1-t), args=(x, t, self.M, self.f))
        #return scipy.optimize.newton(theta_eq, x0, derivative_theta_eq, args=(x, t, self.M, self.f))
    
    def get_theta(x, t):
        if (x,t) not in self.theta:
            self.theta[(x,t)] = self.calculate_theta(x,t)
        return self.theta[(x,t)]
    
    # Define a function that computes p_t_e(x)
    def get_price(m,x,t):
        M = self.M
        f = self.f
        
        if (x,t) not in prices:
            prices[(x,t)] = [None]*M
            for n in constraint_indices:
                prices[(x,t)][n] = t*get_theta(x,t)/(M*(get_theta(x,t) - f(x,n)))
        return prices[(x,t)][m]

# Create (Reduced) LP

In [ ]:
def create_LP(G, multicast_requests):
    nx.set_edge_attributes(G, 1, "weight")
    reduced_LP = gp.Model("Multicast Packing Model - Reduced")
    congestion = reduced_LP.addVar(name="lambda")
    reduced_LP.setObjective(congestion, gp.GRB.MINIMIZE)
    reduced_LP.update()
    # Packing Constraints
    for e in G.edges():
        reduced_LP.addConstr(congestion >= 0, name="{} congestion".format(sorted(e)))
    # Simplex Constraints
    for i in range(len(multicast_requests)):
        reduced_LP.addConstr(0*congestion == 1, name="Tree Selection for {}".format(i))
    reduced_LP.update()
    return reduced_LP

# Column Generating Subproblem

In [ ]:
def cost(G):
        assert nx.is_weighted(G)
        return sum (nx.get_edge_attributes(G, "weight").values())

class MulticastPackingColumnGenerator:
    def __init__(self, instance, reduced_LP):
        self.instance = instance
        self.reduced_LP = reduced_LP
        self.generate_tree = lambda i: nx.algorithms.approximation.steinertree.steiner_tree(self.instance.graph, self.instance.requests[i].multicast_group())
        
    def generate_new_trees(self):
        new_trees = list()
        assert nx.is_weighted(self.instance.graph)
        for i in range(self.instance.num_requests):
            new_tree = self.generate_tree(i)
            new_trees.append(new_tree)
            
            coeffs = list()
            constrs = list()
            # the new tree contributes to the constraint for each of its edges
            for e in new_tree.edges():
                coeffs.append(-1)
                constrs.append(self.reduced_LP.getConstrByName("{} congestion".format(sorted(e))))
            
            # the new tree contributes to the selection constraint for the corresponding multicast request
            coeffs.append(1)
            constrs.append(self.reduced_LP.getConstrByName("Tree Selection for {}".format(i)))
            self.reduced_LP.addVar(name="x{}{}".format(i, nx.info(new_tree)), column=gp.Column(coeffs, constrs))
            
        self.reduced_LP.update()
        return new_trees

# Solvers

## Abstract Solver Class

In [ ]:
class MulticastPackingSolver(ABC):
    
    # Constructor
    def __init__(self, instance=None, block_solver=None):
        # Attributes related to problem instance
        if instance is None:
            instance = MulticastPackingInstance(NUM_MULTICAST_REQUESTS, MAX_MULTICAST_SIZE)
        self.instance = instance
        self.reduced_LP = create_LP(self.instance.graph, self.instance.requests)
        
        # Attributes related to how this solver generates columns and solutions
        if block_solver == None:
            block_solver = MulticastPackingColumnGenerator(self.instance, self.reduced_LP)
        self.column_generator = block_solver
        
        # Attributes that track things
        self.tol = TOLERANCE
        self.stop_flag = False
        self.iteration = 0
        self.solution = list()
        self.objVal = dict()
        self.fVal = dict()
        self.price = dict()
        self.multicast_costs = dict()
        self.new_trees = self.column_generator.generate_new_trees()
        
        if DEBUG_LEVEL >= DEBUG_LEVEL_EXTREME:
            print(reduced_LP.getA().toarray())
            print(reduced_LP.getAttr("RHS"))
    
    # Abstract methods for generating values of functions needed by the solver
    
    @abstractmethod
    def get_next_solution(self):
        pass
    
    @abstractmethod
    def perform_checks_and_updates(self):
        pass
    
    @abstractmethod
    def generate_lamb(self, x):
        pass
    
    @abstractmethod
    def generate_fVal(self, x):
        pass
     
    @abstractmethod   
    def generate_p(self, x, t=0):
        pass
    
    @abstractmethod
    def generate_q(self, x, t=0):
        pass
    
    # Methods for accessing values of functions needed by the solver
    
    def lamb(self, x):
        if x not in self.objVal:
            self.generate_lamb(x)
        return self.objVal[x]
    
    def f(self, x):
        if x not in self.fVal:
            self.generate_fVal(x)
        return self.fVal[x]
    
    def p(self, x, t=0):
        if x not in self.price:
            self.generate_p(x)
        return self.price[x]
    
    def q(self, x, t=0):
        if x not in self.multicast_costs:
            self.generate_q(x)
        return self.multicast_costs[x]
    
    def theta(self, x, t=0):
        return self.lamb(x)
    
    def phi(self, x, t=0):
        return log(self.lamb(x))
    
    def Phi(self, theta, x, t=0):
        return 
    
    def toleranceFunction(self):
        x = self.solution[self.iteration]
        f_prime = dict()
        for e in self.instance.graph.edges():
            f_prime[e] = 0
            for i in range(self.instance.num_requests):
                if self.new_trees[i].has_edge(*e):
                    f_prime[e] += 1
    
        pf = dict_innerProd(self.p(x), self.f(x))
        pf_prime = dict_innerProd(self.p(x), f_prime)
        return (pf-pf_prime)/(pf+pf_prime)
    
    # Main Function
    def perform_iteration(self):
        x  = tuple(self.get_next_solution())
        self.solution.append(x)
        self.generate_lamb(x)
        self.generate_fVal(x)
        self.generate_p(x)
        self.generate_q(x)
        self.new_trees = self.column_generator.generate_new_trees()
            
        self.perform_checks_and_updates(x)
        
        if DEBUG_LEVEL >= DEBUG_LEVEL_THEORY_1:
            print(self.iteration)
            print("lambda(x): {}".format(self.lamb(x)))
            print("phi_t(x): {}".format(self.phi(x)))
            print("tolerance: {}".format(self.toleranceFunction()))
            
        self.iteration += 1

## Pure Column Generation Solver Class

In [ ]:
class PureColGenMcpSolver(MulticastPackingSolver):
    # Constructor
    def __init__(self, instance=None, block_solver=None):
        super().__init__(instance, block_solver)
    
    
    def get_next_solution(self):
        self.reduced_LP.optimize()
        return self.reduced_LP.getVars() # We need to think about how to represent solutions
    
    def perform_checks_and_updates(self, x):
        if ( (sum([cost(self.new_trees[i]) for i in range(self.instance.num_requests)]) >= self.lamb(x))
            or (all([cost(self.new_trees[i]) - self.q(x)[i] >= 0 for i in range(self.instance.num_requests)]))
            or (self.toleranceFunction() <= self.tol) ):
                self.stop_flag = True
                # Could have different stop flags for different stopping criteria
    
    def generate_lamb(self, x):
        self.objVal[x] = self.reduced_LP.getObjective().getValue()
        # Doesn't work if x is not current LP solution
    
    def generate_fVal(self, x):
        self.fVal[x] = dict()
        for e in self.instance.graph.edges():
            self.fVal[x][e] = self.lamb(x) + self.reduced_LP.getConstrByName("{} congestion".format(sorted(e))).getAttr(gp.GRB.Attr.Slack) # Note: Gurobi signs their slacks stupidly
        
    def generate_p(self, x, t=0):
        self.price[x] = dict()
        for e in self.instance.graph.edges():
            self.price[x][e] = self.reduced_LP.getConstrByName("{} congestion".format(sorted(e))).getAttr(gp.GRB.Attr.Pi)
            if (self.price[x][e] < 0):
                print("WHAT IS HAPPENING?")
                print(self.iteration)
                print("lambda(x): {}".format(self.lamb(x)))
                print("log lambda(x): {}".format(log(self.lamb(x))))
                print("tolerance: {}".format(self.toleranceFunction()))
                print("f(x): {}".format(self.f(x)))
                print("p(x): {}".format(self.p(x)))
                print("q(x): {}".format(self.q(x)))
        nx.set_edge_attributes(self.instance.graph, self.price[x], "weight") # I don't think this is how this should be done.
    
    def generate_q(self, x, t=0):
        self.multicast_costs[x] = [None] * len(self.instance.requests)
        for i in range(self.instance.num_requests):
            self.multicast_costs[x][i] = self.reduced_LP.getConstrByName("Tree Selection for {}".format(i)).getAttr(gp.GRB.Attr.Pi)

# Testing

In [ ]:
def test_instance_generation(num_tests):
    assert not is_connected(nx.empty_graph(NUM_NODES))
    assert is_connected(get_random_connected_graph())
    G = get_random_connected_graph()
    for i in range(num_tests):
        request = MulticastRequest(MAX_MULTICAST_SIZE, G)
        source = request.source
        recipients = request.recipients
        assert request.source is not None
        assert request.recipients
        assert not request.source in request.recipients
        assert isinstance(request.recipients, set)
    instance = MulticastPackingInstance(NUM_MULTICAST_REQUESTS, MAX_MULTICAST_SIZE)
    for request in instance.requests:
        assert request.source in instance.graph
        assert request.recipients.issubset(instance.graph)

def test_LP_creation(num_tests):
    instance = MulticastPackingInstance(NUM_MULTICAST_REQUESTS, MAX_MULTICAST_SIZE)
    reduced_LP = create_LP(instance.graph, instance.requests)
    assert len(reduced_LP.getVars()) == 1
    assert len(reduced_LP.getConstrs()) == NUM_EDGES + NUM_MULTICAST_REQUESTS, "Num of constraints: {} Expected: {}".format(len(reduced_LP.getConstrs()), NUM_EDGES + NUM_MULTICAST_REQUESTS)
    
def test_subproblem(num_tests):
    instance = MulticastPackingInstance(NUM_MULTICAST_REQUESTS, MAX_MULTICAST_SIZE)
    reduced_LP = create_LP(instance.graph, instance.requests)
    column_generator = MulticastPackingColumnGenerator(instance, reduced_LP)
    
    new_trees = column_generator.generate_new_trees()
    assert (len(new_trees) == instance.num_requests)
    for tree in new_trees:
        assert(cost(tree) == len(tree.edges()))
           
    assert(len(reduced_LP.getVars()) == 1+instance.num_requests)
    
def run_tests(num_tests):
    test_instance_generation(num_tests)
    test_LP_creation(num_tests)
    test_subproblem(num_tests)

NUM_TESTS = 100
run_tests(NUM_TESTS)

# Run the solver

In [ ]:
for i in range(1):
    print("BEGIN NEW TEST")
    solver = PureColGenMcpSolver()
    while(not solver.stop_flag):
        solver.perform_iteration()
    
    #print("Natural Prices")
    #solve_multicast_packing_via_columngen(*instance, priceFlag=NATURAL)
    #print("Uniform Prices")
    #solve_multicast_packing_via_columngen(*instance, priceFlag=UNIFORM)
    #print("Perturbed Prices")
    #solve_multicast_packing_via_columngen(*instance, priceFlag=PERTURB)